In [2]:
SELECT
    YEAR(OrderDate)     as [Year],
    MONTH(OrderDate)    as [Month],
    COUNT(*)            as [Orders],
    SUM(SubTotal)       as [SubTotal]
FROM Orders
where YEAR(OrderDate) = 2012
GROUP BY YEAR(OrderDate), MONTH(OrderDate)
ORDER BY 1,2

(12 rows affected)

Total execution time: 00:00:00.044

Year,Month,Orders,SubTotal
2012,1,336,"3970627,279"
2012,2,219,"1475426,91"
2012,3,304,"2975748,2385"
2012,4,269,"1634600,7984"
2012,5,293,"3074602,8138"
2012,6,390,"4099354,3572"
2012,7,385,"3417953,8699"
2012,8,285,"2175637,218"
2012,9,352,"3454151,9405"
2012,10,321,"2544091,1056"


In [15]:
SELECT 
    *,
    COUNT(*)        OVER(), --aggregáló ablakozás esetében nem kötelezó az order by
    SUM(Orders)     OVER(partition by Year), --összesíti a rendelést évenként
    SUM(Orders)     OVER(partition by Year order by Month), --göngyölt összeg
    SUM(SubTotal)   OVER(partition by Year order by Month), --göngyölt összeg
    MAX(SubTotal)   OVER(partition by Year order by Month rows between unbounded preceding and current row)


FROM (
    SELECT
        YEAR(OrderDate)     as [Year],
        MONTH(OrderDate)    as [Month],
        COUNT(*)            as [Orders],
        SUM(SubTotal)       as [SubTotal]
    FROM Orders
    where YEAR(OrderDate) in (2012, 2011, 2013)
    GROUP BY YEAR(OrderDate), MONTH(OrderDate)
    --ORDER BY 1,2 Subqueryben nem lehet order by
) as s
ORDER BY 1, 2

(32 rows affected)

Total execution time: 00:00:00.230

Year,Month,Orders,SubTotal,(No column name),(No column name),(No column name),(No column name)
2011,5,43,"503805,9169",32,1607,43,"503805,9169"
2011,6,141,"458910,8248",32,1607,184,"962716,7417"
2011,7,231,"2044600,0034",32,1607,415,"3007316,7451"
2011,8,250,"2495816,7335",32,1607,665,"5503133,4786"
2011,9,157,"502073,8458",32,1607,822,"6005207,3244"
2011,10,327,"4588761,816",32,1607,1149,"10593969,1404"
2011,11,230,"737839,8214",32,1607,1379,"11331808,9618"
2011,12,228,"1309863,2511",32,1607,1607,"12641672,2129"
2012,1,336,"3970627,279",32,3915,336,"3970627,279"
2012,2,219,"1475426,91",32,3915,555,"5446054,189"


Az 5 sor hogyan is működik. Amikor elindul a lekérdezés ugye 1 sor van, ennek az összege 43 mert az ablak 1 soros

a következő iterációban az ablak két soros, így a 1 és 2 összege (43+141) lesz az eredménye.

Ha kiveszed a patrition by-t akkor az ablak az egész lekérdezésre fog vonatkozní, és a göngyölítés a legérdezés végéig <span style="color: var(--vscode-foreground);">megy.</span>

<span style="color: var(--vscode-foreground);">7 Sor</span>

<span style="color: var(--vscode-foreground);">Itt az van definiálva, hogy az ablak folyamatosan nőljön, amíg az év nem változik. Itt lehet olyan, hogy a max érték változik emiatt.</span>

<span style="color: var(--vscode-foreground);">Ha azt szeretnénk, hogy az egész patrition by által megadott ablakban keresse a max-ot, akkor nem kell order by, vagy&nbsp;</span> 

<span style="color: var(--vscode-foreground);">... and unbounded following</span>

<span style="color: var(--vscode-foreground);"><br></span>